In [1]:
# 수정 사항:
# 정렬된 데이터프레임 생성: 방문코드 생성을 위해 df.sort_values()로 정렬한 결과를 df_sorted라는 새로운 데이터프레임에 저장합니다. .copy()를 사용하여 원본 df에 영향을 주지 않도록 합니다.
# 방문코드 및 순번 생성: df_sorted를 기준으로 방문코드와 순번을 생성합니다. 이렇게 하면 "A0000001"은 정렬된 데이터의 첫 번째 방문에 할당됩니다.
# flag, first/last data&time, 자전거대수 계산: 이 단계들도 정렬된 df_sorted를 기준으로 수행합니다.
# 원래 순서 복구 (선택 사항): 만약 최종 결과가 원래 파일의 순서를 유지해야 한다면, 원본 df와 df_sorted를 '자전거번호'와 '날짜+시간'을 기준으로 병합하여 생성된 방문코드 등의 정보를 다시 원본 순서에 맞춥니다.
# 열 순서 정리: 최종 컬럼 순서를 지정합니다.
# 이렇게 수정하면 df_sorted의 첫 번째 행 (정렬 기준에 따른 첫 번째 방문)의 "방문코드"가 "A0000001"이 됩니다. 만약 최종 저장되는 CSV 파일이 원래 순서를 유지해야 한다면, 병합 후에도 첫 번째 행이 "A0000001"을 가지는 방문에 해당할지는 데이터의 특성에 따라 달라질 수 있습니다. 그러나 "방문코드" 컬럼 내에는 "A0000001"이 반드시 포함되게 됩니다.

In [2]:
# 🧹 완전히 클린한 상태로 시작!
import pandas as pd
from datetime import timedelta

# ✅ 1단계: 처음부터 다시 읽기 (덮어쓰기 금지!)
file_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"
df = pd.read_csv(file_path, encoding='utf-8-sig', dtype={
    '대여소': str,
    '대여소명': str,
    '자전거번호': str
})
df['날짜+시간'] = pd.to_datetime(df['날짜+시간'])

# 🔁 혹시라도 남아있을 열들 제거
df.drop(columns=[col for col in df.columns if '방문코드' in col or '순번' in col], errors='ignore', inplace=True)

df['flag'] = ''
df['자전거대수'] = 0
df['first data&time'] = ''
df['last data&time'] = ''

# ✅ 정렬 (방문코드 생성을 위한 정렬)
df_sorted = df.sort_values(by=['대여소', '날짜+시간']).reset_index(drop=True).copy()

# 📌 2단계: 방문코드 및 순번 생성 (A0000001부터 정확히 시작)
visit_index = 1
visit_code = [f"A{visit_index:07d}"]

for i in range(1, len(df_sorted)):
    time_diff = df_sorted.loc[i, "날짜+시간"] - df_sorted.loc[i - 1, "날짜+시간"]
    location_changed = df_sorted.loc[i, "대여소"] != df_sorted.loc[i - 1, "대여소"]
    if time_diff.total_seconds() > 1800 or location_changed:
        visit_index += 1
    visit_code.append(f"A{visit_index:07d}")

df_sorted["방문코드"] = visit_code
df_sorted["순번"] = df_sorted.groupby("방문코드").cumcount() + 1

# 🚩 4단계: flag 생성 (정렬된 데이터프레임 기준)
for i in range(1, len(df_sorted)):
    curr = df_sorted.iloc[i]
    prev = df_sorted.iloc[i - 1]
    if (
        curr['자전거번호'] == prev['자전거번호'] and
        curr['방문코드'] == prev['방문코드'] and
        (curr['날짜+시간'] - prev['날짜+시간']) <= timedelta(minutes=30)
    ):
        df_sorted.at[i, 'flag'] = '가짜'

# ⏱ 5단계: first / last data&time (정렬된 데이터프레임 기준)
for code, group in df_sorted.groupby('방문코드'):
    sorted_group = group.sort_values(by='순번')
    first_idx = sorted_group.index[0]
    last_idx = sorted_group.index[-1]
    df_sorted.at[first_idx, 'first data&time'] = sorted_group.iloc[0]['날짜+시간']
    df_sorted.at[last_idx, 'last data&time'] = sorted_group.iloc[-1]['날짜+시간']

# 📊 6단계: 자전거대수 (정렬된 데이터프레임 기준)
df_sorted['자전거대수'] = df_sorted.groupby('방문코드')['자전거번호'].transform('nunique')

# ✅ 원래 순서 복구 (필요하다면)
df = df.merge(df_sorted[['자전거번호', '날짜+시간', '방문코드', '순번', 'flag', '자전거대수', 'first data&time', 'last data&time']],
                on=['자전거번호', '날짜+시간'], how='left')

# 🧹 열 순서 정리
final_columns = ['No', '자전거번호', '대여소', '대여소명', '날짜', '시간', '날짜+시간',
                 '구분', '방문코드', '순번', 'flag', '자전거대수',
                 'first data&time', 'last data&time']

# 원본 데이터프레임에 병합 후 컬럼 순서 조정
# KeyError 발생 원인: 'No', '날짜', '시간', '구분' 컬럼이 원본 df에 없기 때문입니다.
# 병합된 df에는 해당 컬럼들이 없으므로 final_columns에서 제거해야 합니다.
final_columns_existing = [col for col in final_columns if col in df.columns]
df = df[final_columns_existing]

# 💾 7단계: 저장
output_path = r"D:\pythondata\5-2_회수배송_데이터(2).csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 최종 저장 완료! A0000001부터 시작된 방문코드")
print("📁 저장 경로:", output_path)

✅ 최종 저장 완료! A0000001부터 시작된 방문코드
📁 저장 경로: D:\pythondata\5-2_회수배송_데이터(2).csv
